In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os

BASE1_path = '/kaggle/input/data-chap1-sql-intermediate/'

BASE2_path = '/kaggle/input/data-chap2-sql-intermediate/'

%sql sqlite://

'Connected: @None'

## Loading & viewing dataset

In [2]:
matches_2013_2014 = pd.read_table(os.path.join(BASE1_path, 'matches_2013_2014.txt'), sep = ';')
matches_2013_2014.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,1237,1,2013/2014,1,2014-03-29,8475,9989,2,0
1,1238,1,2013/2014,1,2014-03-29,9991,8573,0,1
2,1239,1,2013/2014,2,2014-04-05,9989,9991,1,0
3,1240,1,2013/2014,2,2014-04-05,8573,8475,0,0
4,1241,1,2013/2014,3,2014-04-12,9991,8475,2,1


In [3]:
team = pd.read_table(os.path.join(BASE2_path, 'team.txt'), sep = ';')
team.head()

,id,team_api_id,team_long_name,team_short_name
0,1,9987,KRC Genk,GEN
1,2,9993,Beerschot AC,BAC
2,3,10000,SV Zulte-Waregem,ZUL
3,4,9994,Sporting Lokeren,LOK
4,5,9984,KSV Cercle Brugge,CEB


In [4]:
match = pd.read_table(os.path.join(BASE1_path, 'match.txt'), sep = ';')
match.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,757,1,2011/2012,1,2011-07-29,1773,8635,2,1
1,758,1,2011/2012,1,2011-07-30,9998,9985,1,1
2,759,1,2011/2012,1,2011-07-30,9987,9993,3,1
3,760,1,2011/2012,1,2011-07-30,9991,9984,0,1
4,761,1,2011/2012,1,2011-07-30,9994,10000,0,0


In [5]:
country = pd.read_table(os.path.join(BASE1_path, 'EU_country.txt'), sep = ';')
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [6]:
league = pd.read_table(os.path.join(BASE2_path, 'league.txt'), sep = ';')
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


## create table

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////football_EU_league', echo=False)

match.to_sql('match', con = engine)
matches_2013_2014.to_sql('matches_2013_2014', con = engine)
country.to_sql('country', con = engine)
team.to_sql('team', con = engine)
league.to_sql('league', con = engine)

# PRACTICEs
## 1. OVER function / clause

### 1.1. The match is OVER

The **`OVER()`** clause allows you to pass an aggregate function down a data set, similar to subqueries in **SELECT**. The **OVER()** clause offers significant benefits over subqueries in select -- namely, your queries will run faster, and the **OVER()** clause has a wide range of additional functions and clauses you can include with it that we will cover later on in this chapter.

In this exercise, you will revise some queries from previous chapters using the **OVER()** clause.

#### Instructions

Select the match `ID`, `country name`, `season`, `home`, and `away goals` from the `match` and `country` tables.

Complete the query that **calculates the average number** of goals scored overall and then includes the aggregate value in each row using a `window function`.

In [8]:
pd.read_sql(
    """ 
        SELECT m.id, c.name AS country, 
               m.season, m.home_goal, m.away_goal,
        
            -- Use a window to include the aggregate average in each row
            AVG(m.home_goal + m.away_goal) OVER() AS overall_avg

        FROM match AS m
        LEFT JOIN country AS c 
            ON m.country_id = c.id
        
        LIMIT 100;    
    """, con = engine)

,id,country,season,home_goal,away_goal,overall_avg
0,757,Belgium,2011/2012,2,1,2.732103
1,758,Belgium,2011/2012,1,1,2.732103
2,759,Belgium,2011/2012,3,1,2.732103
3,760,Belgium,2011/2012,0,1,2.732103
4,761,Belgium,2011/2012,0,0,2.732103
...,...,...,...,...,...,...
95,852,Belgium,2011/2012,1,1,2.732103
96,853,Belgium,2011/2012,3,0,2.732103
97,854,Belgium,2011/2012,6,1,2.732103
98,855,Belgium,2011/2012,0,0,2.732103


The basic window function with an **OVER** clause functions just like a `subquery` in **SELECT**.

### 1.2. What's OVER here?
`Window functions` allow you to create a **RANK** of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an **ORDER BY** clause inside the **OVER()** clause. Below is an example:

                SELECT 
                    id,
                    RANK() OVER(ORDER BY home_goal) AS rank
                FROM match;
In this exercise, you will create a data set of ranked matches according to which leagues, on average, score the most goals in a match.

#### Instructions
Select the `league` `name` and `average total goals` scored from `league` and `match`.

Complete the `window function` so it calculates the rank of average goals scored across all leagues in the database.

Order the `rank` by the `average` total of home and away goals scored.

In [9]:
pd.read_sql(
    """ 
        SELECT l.name AS league,
               AVG(m.home_goal + m.away_goal) AS avg_goals,
               
               -- Rank each league according to the average goals
               RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank
       
        FROM league AS l
        
        LEFT JOIN match AS m 
                ON l.id = m.country_id
        WHERE m.season = '2011/2012'
        
        GROUP BY l.name
        ORDER BY league_rank;    
    """, con = engine)

,league,avg_goals,league_rank
0,Poland Ekstraklasa,2.195833,1
1,France Ligue 1,2.515789,2
2,Italy Serie A,2.583799,3
3,Switzerland Super League,2.623457,4
4,Scotland Premier League,2.635965,5
5,Portugal Liga ZON Sagres,2.641667,6
6,Spain LIGA BBVA,2.763158,7
7,England Premier League,2.805263,8
8,Germany 1. Bundesliga,2.859477,9
9,Belgium Jupiler League,2.879167,10


You'll notice that the default order is from `smallest` to `largest` value -- in this case, the **`Polish league`** tends to have the fewest goals scored on average.

### 1.3. Flip OVER your results
In the last exercise, the rank generated in your query was organized from smallest to largest. By adding **DESC** to your window function, you can create a rank sorted from largest to smallest.

            SELECT 
                id,
                RANK() OVER(ORDER BY home_goal DESC) AS rank
            FROM match;
#### Instructions
Complete the same parts of the query as the previous exercise.

Complete the `window function` to rank each `league` from highest to lowest average goals scored.

Order the main query by the rank you just created.

In [10]:
pd.read_sql(
    """ 
        SELECT l.name AS league,
               AVG(m.home_goal + m.away_goal) AS avg_goals,
               -- Rank leagues in descending order by average goals
               RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal) DESC) AS league_rank

        FROM league AS l

        LEFT JOIN match AS m 
            ON l.id = m.country_id

        WHERE m.season = '2011/2012'
        GROUP BY l.name
        ORDER BY league_rank;    
    """, con = engine)

,league,avg_goals,league_rank
0,Netherlands Eredivisie,3.258170,1
1,Belgium Jupiler League,2.879167,2
2,Germany 1. Bundesliga,2.859477,3
3,England Premier League,2.805263,4
4,Spain LIGA BBVA,2.763158,5
5,Portugal Liga ZON Sagres,2.641667,6
6,Scotland Premier League,2.635965,7
7,Switzerland Super League,2.623457,8
8,Italy Serie A,2.583799,9
9,France Ligue 1,2.515789,10


Unlike a `subquery` in **SELECT**, your `window function` will apply the filter that you include in your **`WHERE`** `clause`.

## 2. OVER with a PARTITION
### 2.1. PARTITION BY a column
The **PARTITION BY** clause allows you to calculate separate `"windows"` based on columns you want to divide your results. For example, you can create a single column that calculates an overall average of goals scored for each season.

In this exercise, you will be creating a data set of games played by `Legia Warszawa (Warsaw League)`, the top ranked team in `Poland`, and comparing their individual game performance to the overall average for that season.

Where do you see more outliers? Are they `Legia Warszawa's home` or `away` games?

#### Instructions
Complete the two `window functions` that calculate the `home` and `away goal` averages. Partition the `window functions` by `season` to calculate separate averages for each season.

Filter the query to only include matches played by `Legia Warszawa`, `id = 8673`.

In [11]:
pd.read_sql(
    """ 
        SELECT date, season, home_goal, away_goal,
            CASE WHEN hometeam_id = 8673 THEN 'home' 
                 ELSE 'away' END AS warsaw_location,


            -- Calculate the average goals scored partitioned by season
            AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
            AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg

        FROM match

        WHERE (hometeam_id = 8673) OR (awayteam_id = 8673)
            ORDER BY (home_goal + away_goal) DESC
        
        LIMIT 100;    
    """, con = engine)

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-09-14,2013/2014,3,5,away,1.766667,1.233333
1,2014-09-13,2014/2015,4,3,home,1.566667,1.333333
2,2013-07-20,2013/2014,5,1,home,1.766667,1.233333
3,2013-10-20,2013/2014,4,1,home,1.766667,1.233333
4,2013-06-02,2012/2013,5,0,home,1.566667,1.133333
...,...,...,...,...,...,...,...
95,2014-03-29,2013/2014,1,0,home,1.766667,1.233333
96,2011-10-16,2011/2012,0,1,away,1.033333,0.933333
97,2013-08-24,2013/2014,0,1,home,1.766667,1.233333
98,2013-08-31,2013/2014,0,1,away,1.766667,1.233333


You've generated an ordered data set with window functions to compare each game's score to averages for that season.

### 2.2. PARTITION BY multiple columns
The **PARTITION BY** `clause` can be used to **break out window averages by multiple data points (columns)**. 

You can even calculate the information you want to use to `partition`your data! For example, you can calculate average goals scored by season and by `country`, or by the `calendar year` (taken from the `date` column).

In this exercise, you will calculate the average number home and away goals scored `Legia Warszawa`, and their opponents, partitioned by the month in each season.

#### Instructions
Construct two `window functions` partitioning the average of `home` and `away` goals by `season` and `month`.

Filter the data set by `Legia Warszawa`'s `team ID (8673)` so that the `window` calculation excludes all teams who did not play against them.

In [12]:
pd.read_sql(
    """ 
        SELECT date, season, home_goal, away_goal,
            CASE WHEN hometeam_id = 8673 THEN 'home' 
                 ELSE 'away' END AS warsaw_location,

            -- Calculate average goals partitioned by season and month
            AVG(home_goal) OVER(PARTITION BY season, 
                                 EXTRACT(month FROM date)
                                 ) AS season_mo_home,
            AVG(away_goal) OVER(PARTITION BY season, 
                                 EXTRACT(month FROM date)
                                 ) AS season_mo_away
        FROM match
        WHERE (hometeam_id = 8673) OR awayteam_id = 8673
        ORDER BY (home_goal + away_goal) DESC
        LIMIT 100;    
    """, con = engine)

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-09-14,2013/2014,3,5,away,2.250000,2.500000
1,2014-09-13,2014/2015,4,3,home,2.000000,2.666667
2,2013-07-20,2013/2014,5,1,home,2.500000,2.000000
3,2014-08-09,2014/2015,5,0,home,2.000000,1.000000
4,2012-10-28,2012/2013,3,2,home,1.666667,2.000000
...,...,...,...,...,...,...,...
95,2012-12-07,2012/2013,1,0,away,2.000000,0.000000
96,2013-05-05,2012/2013,1,0,home,0.600000,0.800000
97,2014-07-19,2014/2015,0,1,home,0.500000,2.000000
98,2013-11-10,2013/2014,0,1,away,1.666667,0.666667


You successfully partitioned your averages by each month in the season. In the next lesson, we'll expand on the usefulness of information generated with window functions.

## 3. Sliding Windows
### 3.1. Slide to the left
**Sliding windows** allow you to create running calculations between any two points in a window using functions such as **PRECEDING**, **FOLLOWING**, and **CURRENT ROW**. You can calculate running `counts, sums, averages`, and `other aggregate functions` between **`any two points`** you specify in the data set.

In this exercise, you will expand on the examples discussed in the video, calculating the running total of goals scored by the FC Utrecht when they were the home team during the 2011/2012 season. Do they score more goals at the end of the season as the home or away team?

#### Instructions
Complete the `window function` by:

Assessing the `running total` of home goals scored by `FC Utrecht`.

Assessing the `running average` of home goals scored.

Ordering both the running average and running total by `date`.

In [13]:
pd.read_sql(
    """ 
        SELECT date, home_goal, away_goal,
            -- Create a running total and running average of home goals
            SUM(home_goal) OVER(ORDER BY date
                                 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                                 ) AS running_total,
            AVG(home_goal) OVER(ORDER BY date
                                 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                                 ) AS running_avg
        FROM match
        WHERE (hometeam_id = 9908) AND (season = '2011/2012')
    """, con = engine)

,date,home_goal,away_goal,running_total,running_avg
0,2011-08-14,2,2,2,2.000000
1,2011-08-27,3,1,5,2.500000
2,2011-09-18,2,2,7,2.333333
3,2011-10-01,3,0,10,2.500000
4,2011-10-22,1,4,11,2.200000
5,2011-11-06,6,4,17,2.833333
6,2011-12-04,2,6,19,2.714286
7,2011-12-11,2,2,21,2.625000
8,2012-01-22,1,1,22,2.444444
9,2012-02-12,1,1,23,2.300000


Sliding windows allow for a wide variety of calculations with your database.

### 3.2. Slide to the right
Now let's see how `FC Utrecht` performs when they're the `away team`. You'll notice that the total for the season is at the bottom of the data set you queried. Depending on your results, this could be pretty long, and scrolling down is not very helpful.

In this exercise, you will slightly modify the query from the previous exercise by sorting the data set in reverse order and **calculating a backward running** total from the **CURRENT ROW** to the end of the data set (`earliest record`).

#### Instructions
Complete the window function by:

Assessing the `running total` of home goals scored by `FC Utrecht`.

Assessing the `running average` of home goals scored.

Ordering both the `running average` and running `total` by `date`, `descending`.

In [14]:
pd.read_sql(
    """ 
        SELECT date, home_goal, away_goal,

               -- Create a running total and running average of home goals
               SUM(home_goal) OVER(ORDER BY date DESC
                                     ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
                                     ) AS running_total,
               AVG(home_goal) OVER(ORDER BY date DESC
                                     ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
                                     ) AS running_avg
        FROM match
        WHERE (awayteam_id = 9908) AND (season = '2011/2012');    
    """, con = engine)

,date,home_goal,away_goal,running_total,running_avg
0,2012-05-06,1,3,25,1.470588
1,2012-04-21,0,2,24,1.500000
2,2012-04-12,3,0,24,1.600000
3,2012-03-25,3,1,21,1.500000
4,2012-03-11,1,1,18,1.384615
5,2012-02-26,1,0,17,1.416667
6,2012-02-05,0,2,16,1.454545
7,2012-01-28,2,0,16,1.600000
8,2011-12-17,1,0,14,1.555556
9,2011-11-25,2,0,13,1.625000


You can reverse your calculations with your window functions to best suit your needs in an analysis.

## 4. Bringing it all together
### 4.1 Setting up the home team CTE
In this course, we've covered ways in which you can use **CASE** `statements`, subqueries, common table expressions, and window functions in your queries to structure a data set that best meets your needs. For this exercise, you will be using all of these concepts to generate a list of matches in which `Manchester United` was defeated during the **`2014/2015 English Premier League season`**.

Your first task is to create the first query that filters for matches where `Manchester United` played as the home team. This will become a common table expression in a later exercise.

#### Instructions
Create a **CASE** statement that identifies each match as a `win`, `lose`, or `tie` for `Manchester United`.

Fill out the logical operators for each **WHEN** clause in the **CASE** statement (`equals`, `greater than`, `less than`).

Join the tables on `home team ID` from `match`, and `team_api_id` from `team`.

Filter the query to only include games from the `2014/2015 season` where `Manchester United` was the `home team`.

In [15]:
pd.read_sql(
    """ 
        SELECT m.id, t.team_long_name,
            -- Identify matches as home/away wins or ties
            CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
                 WHEN m.home_goal < m.away_goal THEN 'MU Loss'
                 ELSE 'Tie' END AS outcome
        FROM match AS m

        LEFT JOIN team AS t 
            ON m.hometeam_id = t.team_api_id
        WHERE (season = '2014/2015') AND (t.team_long_name = 'Manchester United');    
    """, con = engine)

,id,team_long_name,outcome
0,4013,Manchester United,MU Loss
1,4031,Manchester United,MU Win
2,4051,Manchester United,MU Win
3,4062,Manchester United,MU Win
4,4085,Manchester United,MU Win
5,4105,Manchester United,MU Win
6,4145,Manchester United,MU Loss
7,4164,Manchester United,MU Win
8,4181,Manchester United,MU Win
9,4203,Manchester United,MU Win


This query you wrote will become the first common table expression.

### 4.2. Setting up the away team CTE
Great job! Now that you have a query identifying the home team in a match, you will perform a similar set of steps to identify the away team. Just like the previous step, you will join the match and team tables. Each of these two queries will be declared as a **Common Table Expression (CTEs)** in the following step.

The primary difference in this query is that you will be joining the tables on awayteam_id, and reversing the match outcomes in the **CASE** `statement`.

When altering **CASE** `statement` logic in your own work, you can reverse either the logical condition (i.e., `home_goal > away_goal`) or the outcome in **THEN** -- just make sure you only reverse one of the two!

#### Instructions
Complete the **CASE** statement syntax.

Fill out the logical operators identifying each match as a `win`, `loss`, or `tie` for `Manchester United`.

Join the table on `awayteam_id`, and `team_api_id`.

In [16]:
pd.read_sql(
    """ 
        SELECT m.id, t.team_long_name,
            -- Identify matches as home/away wins or ties
            CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
                 WHEN m.home_goal < m.away_goal THEN 'MU Win'
                 ELSE 'Tie' END AS outcome

        FROM match AS m
        LEFT JOIN team AS t 
            ON m.awayteam_id = t.team_api_id
        WHERE (season = '2014/2015') AND (t.team_long_name = 'Manchester United');    
    """, con = engine)

,id,team_long_name,outcome
0,4026,Manchester United,MU Loss
1,4039,Manchester United,MU Win
2,4075,Manchester United,MU Win
3,4089,Manchester United,Tie
4,4117,Manchester United,Tie
5,4126,Manchester United,Tie
6,4136,Manchester United,Tie
7,4155,Manchester United,MU Win
8,4178,Manchester United,Tie
9,4197,Manchester United,MU Loss


You correctly constructed a **CASE** statement to accurately identify matches as a `win` or `loss` for `Manchester United`. This will become your second **`common table expression`**.

### 4.3 Putting the CTEs together
Now that you've created the two `subqueries` identifying the home and away team opponents, it's time to rearrange your query with the home and away subqueries as **Common Table Expressions (CTEs)**. You'll notice that the main query includes the phrase, **SELECT DISTINCT**. Without identifying only **DISTINCT** matches, you will return a duplicate record for each game played.

Continue building the query to extract all matches played by `Manchester United` in the `2014/2015 season`.

#### Instructions
Declare the `home` and `away` **CTEs** before your `main query`.

Join your **CTEs** to the match table using a **LEFT JOIN**.

Select the `relevant` data from the **CTEs** into the `main query`.

Select the date from `match`, `team names` from the **CTEs**, and `home/ away goals` from `match` in the `main query`.

In [17]:
pd.read_sql(
    """ 
        -- Set up the home team CTE
        WITH home AS ( SELECT m.id, t.team_long_name,
                       CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
                            WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
                            ELSE 'Tie' 
                        END AS outcome
                      FROM match AS m
                      LEFT JOIN team AS t 
                          ON m.hometeam_id = t.team_api_id),

        -- Set up the away team CTE
        away AS ( SELECT m.id, t.team_long_name,
                  CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
                       WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
                       ELSE 'Tie' 
                  END AS outcome
          FROM match AS m
          LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)

        -- Select team names, the date and goals
        SELECT DISTINCT m.date,
                        home.team_long_name AS home_team,
                        away.team_long_name AS away_team,
                        m.home_goal, m.away_goal

        -- Join the CTEs onto the match table
        FROM match AS m
            LEFT JOIN home ON m.id = home.id
                LEFT JOIN away ON m.id = away.id
                
        WHERE m.season = '2014/2015'
              AND (home.team_long_name = 'Manchester United' 
                   OR away.team_long_name = 'Manchester United');    
    """, con = engine)

,date,home_team,away_team,home_goal,away_goal
0,2014-08-16,Manchester United,Swansea City,1,2
1,2014-08-24,Sunderland,Manchester United,1,1
2,2014-08-30,Burnley,Manchester United,0,0
3,2014-09-14,Manchester United,Queens Park Rangers,4,0
4,2014-09-21,Leicester City,Manchester United,5,3
5,2014-09-27,Manchester United,West Ham United,2,1
6,2014-10-05,Manchester United,Everton,2,1
7,2014-10-20,West Bromwich Albion,Manchester United,2,2
8,2014-10-26,Manchester United,Chelsea,1,1
9,2014-11-02,Manchester City,Manchester United,1,0


You wrote a query that extracted data from `two common table expressions`. You can scale up the number of `CTEs` as necessary, depending on your data.

### 4.4. Add a window function
Fantastic! You now have a result set that retrieves the match date, home team, away team, and the goals scored by each team. You have one final component of the question left -- **how badly did Manchester United lose in each match?**

In order to determine this, let's add a window function to the main query that ranks matches by the `absolute` value of the `difference` between `home_goal` and `away_goal`. This allows us to directly compare the difference in scores without having to consider whether `Manchester United` played as the `home` or `away` team!

The equation is complete for you -- all you need to do is properly complete the `window function`!

#### Instructions
Set up the **CTEs** so that the home and away teams each have a `name`, `ID`, and `score` associated with them.

Select the `date`, `home team name`, `away team name`, `home goal`, and `away goals` scored in the `main query`.

Rank the matches and order by the `difference` in scores in `descending` order.

In [18]:
pd.read_sql(
    """ 
        -- Set up the home team CTE
        WITH home AS ( SELECT m.id, t.team_long_name,
                       CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
                            WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
                            ELSE 'Tie' 
                       END AS outcome
                       FROM match AS m

          LEFT JOIN team AS t 
              ON m.hometeam_id = t.team_api_id),

        -- Set up the away team CTE
        away AS ( SELECT m.id, t.team_long_name,
                  CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
                       WHEN m.home_goal < m.away_goal THEN 'MU Win' 
                       ELSE 'Tie' 
                  END AS outcome
                  FROM match AS m

          LEFT JOIN team AS t 
              ON m.awayteam_id = t.team_api_id)

        -- Select columns and and rank the matches by date
        SELECT DISTINCT m.date,
                        home.team_long_name AS home_team,
                        away.team_long_name AS away_team,
                        m.home_goal, m.away_goal,
                        RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) AS match_rank

        -- Join the CTEs onto the match table
        FROM match AS m
            LEFT JOIN home ON m.id = home.id
                LEFT JOIN AWAY ON m.id = away.id
                
        WHERE m.season = '2014/2015'
              AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
              OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));    
    """, con = engine)

,date,home_team,away_team,home_goal,away_goal,match_rank
0,2014-08-16,Manchester United,Swansea City,1,2,3
1,2014-09-21,Leicester City,Manchester United,5,3,2
2,2014-11-02,Manchester City,Manchester United,1,0,3
3,2015-01-11,Manchester United,Southampton,0,1,3
4,2015-02-21,Swansea City,Manchester United,2,1,3
5,2015-04-18,Chelsea,Manchester United,1,0,3
6,2015-04-26,Everton,Manchester United,3,0,1
7,2015-05-02,Manchester United,West Bromwich Albion,0,1,3
